The set of PSD matrices
$$\mathcal{S}^n_+ := \{ X \in \mathcal{S} : z^T X z \geq 0 \ \forall z \in \mathbb{R}^n \},$$
where $\mathcal{S}^n$ is the set of symmetric n by n matrices, forms a convex cone.

If symmetric matrix $Q$ has eigenvalues $\lambda_1, \lambda_2 \ldots \lambda_n$, then $tI-Q$ has eigenvalues $t-\lambda_1, t-\lambda_2 \ldots t-\lambda_n$.  $tI-Q$ is PSD exactly when all its eignevalues are non-negative, which is true only when $t \geq \lambda_1$ its largest eigenvalue.  Thus, we can model the problem of finding the largest eigenvalue of a symmetric matrix $Q$ with the following optimization problem 

$$\lambda_1 = \min t : tI - Q \succeq 0$$

In [ ]:
# This one will only work with SCS optimizer. 
using JuMP, LinearAlgebra, SCS 

A = [3 2 4; 2 0 2; 4 2 3]
I = Matrix{Float64}(LinearAlgebra.I, 3, 3)
model = Model(SCS.Optimizer)
set_silent(model)
@variable(model, t)
@objective(model, Min, t)
@constraint(model, t .* I - A in PSDCone())
optimize!(model)
@assert is_solved_and_feasible(model)
objective_value(model)

Let's see an SDP where we minimize the amount added to the diagonal of a symmetric (non-PSD) matrix in order to make it PSD 

In [ ]:
using LinearAlgebra 
Q = [2 4 -2 -1; 4 -1 3 0; -2 3 0 0; -1 0 0 3]
eig = eigen(Q)
print("Min eigenvalue is ", minimum(eig.values))

# Now lets find the smallest amount to Q to make it PSD 
model = Model(SCS.Optimizer)
@variable(model, X[1:4, 1:4])
@objective(model, Min, sum(X[j,j] for j in 1:4))

for i in 1:4
    for j in 1:4
        if i != j
            @constraint(model, X[i,j] == 0)
        end 
    end 
end 

@constraint(model, (X + Q) in PSDCone())

#print(model)

optimize!(model)
@assert is_solved_and_feasible(model)
print(objective_value(model))
value.(X)